# Лабораторная №1. Проверка статистических гипотез

> Вариант № ??

**Распределения**:

$X_1$ ~ ?? (объём выборки $n_1$ = ??)

$X_2$ ~ ?? (объём выборки $n_2$ = ??)


*Импорт необходимых библиотек:*

In [1]:
import numpy as np
import matplotlib as plt
import scipy as sp

## Генерация выборок

`x = np.random.<`*`distribution`*`>(`*`...params`*`, `*`size`*`)`, где

- *distribution* - распределение;
- *...params* - параметры распределения;
- *size* - размер выборки

Доступные распределения:

- нормальное: normal(m, $\sigma^2$)
- равномерное: uniform(a, b)
- хи-квадрат: chisquare(k)


In [2]:
# Sample sizes
n1 = 100
n2 = 100
n3 = 50

# Samples
x1 = np.random.chisquare(3, n1)
x2 = np.random.normal(0, 1, n2)
x3 = np.random.uniform(-1, 1, n3)

In [3]:
print(np.mean(x1))
print(np.mean(x2))
print(np.mean(x3))

3.07118488769
0.0150836974397
0.035739355966
